In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from tqdm import tqdm
import torchnet as tnt
from torchnet.engine import Engine
from torch.autograd import Variable
from torch.optim import Adam

class RadioTransformerNetworkRNN(nn.Module):
    def __init__(self, in_channels, compressed_dim, hidden_size, num_layers):
        super(RadioTransformerNetworkRNN, self).__init__()

        self.in_channels = in_channels
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.encoder = nn.RNN(
            input_size=in_channels, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True
        )

        # Decoder RNN
        self.decoder = nn.RNN(
            input_size=compressed_dim, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True
        )

        self.fc_encoder = nn.Linear(hidden_size, compressed_dim)
        self.fc_decoder = nn.Linear(hidden_size, in_channels)

    def encode_signal(self, x):
        out, _ = self.encoder(x)
        out = self.fc_encoder(out[:, -1, :])  # Take the last time step
        return out

    def decode_signal(self, x):
        x = x.unsqueeze(1).repeat(1, self.in_channels, 1)  # Repeat for sequence length
        out, _ = self.decoder(x)
        out = self.fc_decoder(out[:, -1, :])
        return out

    def forward(self, x):
        # Encode
        x, _ = self.encoder(x)  # x: [batch_size, seq_len, hidden_size]
        x = self.fc_encoder(x[:, -1, :])  # Take the last time step
    
        # Normalization
        x = (self.in_channels ** 2) * (x / x.norm(dim=-1, keepdim=True))
    
        # Add Gaussian noise
        training_signal_noise_ratio = 5.01187
        communication_rate = 1
        noise = torch.randn_like(x) / ((2 * communication_rate * training_signal_noise_ratio) ** 0.5)
        if USE_CUDA:
            noise = noise.cuda()
        x += noise
    
        # Decode
        x = x.unsqueeze(1).repeat(1, self.in_channels, 1)  # shape: [batch_size, seq_len, compressed_dim]
        x, _ = self.decoder(x)  # x: [batch_size, seq_len, hidden_size]
        x = self.fc_decoder(x[:, -1, :])  # Take the last time step
    
        return x


# Iterator and other utility functions

def get_iterator(mode):
    data = train_data if mode else test_data
    labels = train_labels if mode else test_labels
    tensor_dataset = tnt.dataset.TensorDataset([data, labels])

    return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)

def processor(sample):
    data, labels, training = sample

    data = Variable(data)
    labels = Variable(labels)

    if USE_CUDA:
        data = data.cuda()
        labels = labels.cuda()

    outputs = model(data)

    loss = loss_fn(outputs, labels)

    return loss, outputs

def reset_meters():
    meter_accuracy.reset()
    meter_loss.reset()
    confusion_meter.reset()

def on_sample(state):
    state['sample'].append(state['train'])

def on_forward(state):
    meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    meter_loss.add(state['loss'].item())

def on_start_epoch(state):
    reset_meters()
    state['iterator'] = tqdm(state['iterator'])

def on_end_epoch(state):
    print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    reset_meters()

    engine.test(processor, get_iterator(False))

    print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

import math
# Model Parameters
NUM_EPOCHS = 100
BATCH_SIZE = 256
CHANNEL_SIZE = 100
USE_CUDA = True
HIDDEN_SIZE = 64
NUM_LAYERS = 2
COMPRESSED_DIM = int(math.log2(CHANNEL_SIZE))

# Model Initialization
model = RadioTransformerNetworkRNN(
    in_channels=CHANNEL_SIZE,
    compressed_dim=COMPRESSED_DIM,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS
)

if USE_CUDA:
    model = model.cuda()

# Example Data Initialization
train_labels = (torch.rand(10000) * CHANNEL_SIZE).long()
train_data = torch.eye(CHANNEL_SIZE).index_select(dim=0, index=train_labels).unsqueeze(1)  # Add sequence dimension

test_labels = (torch.rand(1500) * CHANNEL_SIZE).long()
test_data = torch.eye(CHANNEL_SIZE).index_select(dim=0, index=test_labels).unsqueeze(1)  # Add sequence dimension

# Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Meters for tracking performance
meter_loss = tnt.meter.AverageValueMeter()
meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
confusion_meter = tnt.meter.ConfusionMeter(CHANNEL_SIZE, normalized=True)
engine = Engine()

loss_fn = nn.CrossEntropyLoss()

In [3]:
engine.hooks['on_sample'] = on_sample
engine.hooks['on_forward'] = on_forward
engine.hooks['on_start_epoch'] = on_start_epoch
engine.hooks['on_end_epoch'] = on_end_epoch

engine.train(processor, get_iterator(True), maxepoch=NUM_EPOCHS, optimizer=optimizer)

100%|██████████| 40/40 [00:01<00:00, 28.39it/s]

[Epoch 1] Training Loss: 4.5151 (Accuracy: 2.92%)


[Epoch 1] Testing Loss: 4.2682 (Accuracy: 6.67%)


100%|██████████| 40/40 [00:00<00:00, 41.50it/s]

[Epoch 2] Training Loss: 3.9980 (Accuracy: 7.42%)


...

[Epoch 99] Testing Loss: 0.0048 (Accuracy: 100.00%)


100%|██████████| 40/40 [00:00<00:00, 44.56it/s]

[Epoch 100] Training Loss: 0.0047 (Accuracy: 100.00%)


[Epoch 100] Testing Loss: 0.0046 (Accuracy: 100.00%)


{'network': <function __main__.processor(sample)>,
 'iterator': <tqdm.std.tqdm at 0x7f95aedfc250>,
 'maxepoch': 100,
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 'epoch': 100,
 't': 4000,
 'train': True,
 'sample': [tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.]],
  
          ...,
  
          [[0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.]]]),
  tensor([32, 27, 33, 89, 47, 20, 76, 57, 46, 29, 93, 25, 73, 61, 45, 54]),
  True],
 'output': None,
 'loss': None}